### TODO
1. ~~Code up VAE~~
2. ~~Code up M (RNN)~~
3. Code up Train Loop for M
3. Code up C (Controller)
4. Set up Car Racing
5. Set up Doom

In [ ]:
import torch

import numpy as np

from torch import nn

from torch.autograd import Variable

from torch.nn import functional as F
from torch.nn import init
import time
from torch import optim
from copy import deepcopy
from tqdm import tqdm
from matplotlib import pyplot as plt
from tensorboardX import SummaryWriter
import os

import sys

if "ipykernel_launcher" in sys.argv[0]:
    sys.argv = [""]

In [ ]:
import gym

import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--weights_file",type=str,default="None")
parser.add_argument("--lr",type=float, default=0.0001)
parser.add_argument("--ctl_type",type=str, default="lstm")
parser.add_argument("--opt",type=str, default="adam")
parser.add_argument("--iters",type=int, default=100000)
args = parser.parse_args()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Imports specifically so we can render outputs in Jupyter.
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display

def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    #plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    display(display_animation(anim, default_mode='loop'))

In [ ]:
def data_gen(bs=128,nb=1):
    for _ in range(nb):
        x = Variable(torch.Tensor(bs,3,64,64).uniform_(-1,1)).cuda()
        yield x

In [ ]:
def get_optim(name,  model, lr, momentum):
    if name == "adam":
        return optim.Adam(params=model.parameters(),
                        lr=lr)
    elif name == "sgd":
        return optim.SGD(params=model.parameters(),
                        lr=lr,
                        momentum=momentum)
    elif name == "rmsprop":
          return optim.RMSprop(params=model.parameters(),
                        lr=lr,
                        momentum=momentum)

def print_info(mode,loss,t0,it):
    print("time: %8.4f"% (time.time() - t0))
    print("%s Loss for it %i: %8.4f"%(mode.capitalize(),it,loss))
    #print("%s Accuracy for epoch %i: %8.4f"%(mode.capitalize(),epoch,acc))

In [8]:
def do_epoch(mode, model,opt,epoch):
    criterion = vae_loss
    dataloader = data_gen()
    if mode == "train":
        model.train()
    else:
        model.eval()
    it_losses = []
    for it, xv in enumerate(dataloader):

        if mode == "train":
            opt.zero_grad()
        xh,mu,sigma = model(xv)

        loss = criterion(xv,xh,mu,sigma)

        it_losses.append(loss.data[0])
        if mode == "train":
            loss.backward()
            opt.step()


    loss = np.mean(it_losses)
    #writer.add_scalar("%s/loss"%mode,scalar_value=loss,global_step=epoch)
    return loss

def train(
            model,
            lr=0.1,
            momentum=0.9,
            test=False,
            val=True,
            verbose=True,
            num_epochs=5, optimizer="adam"):
    
        if test:
            modes = ["test"]
            num_epochs = 1
            if args.weights_file == "None":
                assert False, "I don't think you meant to run on the test set without a weights file!"
        else:
            modes = ["train"]
            if val:
                modes.append("val")
        
        if args.weights_file != "None":
            model.load_state_dict(torch.load(args.weights_file))


        model = model.cuda()
        opt = get_optim(optimizer,model,lr,momentum)
        for epoch in range(num_epochs):
            for mode in modes:
                t0 = time.time()
                loss = do_epoch(mode,model,opt,epoch)
                if verbose:
                    print_info(mode,loss,t0,epoch)
#                 if mode == "train":
#                     torch.save(model.state_dict(), saved_model_dir +'/epoch_%i.pt' % epoch)

In [4]:
env = gym.make('CarRacing-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [210]:
class M(nn.Module):
    def __init__(self,env="CarRacing"):
        super(M,self).__init__()
        if env == "CarRacing":
            self.nz = 32
            self.nh = 256
            self.action_len = 3 #3 continuous values
        elif env == "Doom":
            pass
#             self.nz = 64
#             self.nh = 512

        self.num_gaussians = 5
        
        self.sigma_len = self.nz
        self.mu_len = self.nz
        self.pi_len = 1
        self.len_mdp_output = 5*(self.sigma_len + self.mu_len + self.pi_len)
        

        self.rnn = nn.LSTM(input_size=self.nz+self.action_len,
                           hidden_size=self.nh,num_layers=1)
        
        self.mdn_fc = nn.Linear(in_features=self.nh,
                                out_features=self.len_mdp_output)
        
        self.h_prev = Variable(torch.Tensor(1,batch_size,m.nh).normal_()).cuda()
        self.c_prev = Variable(torch.Tensor(1,batch_size,m.nh).normal_()).cuda()
    
    def reset(self):
        self.h_prev = Variable(torch.Tensor(1,batch_size,m.nh).normal_()).cuda()
        self.c_prev = Variable(torch.Tensor(1,batch_size,m.nh).normal_()).cuda()
    
    def postproc_mdp_out(self,mdp_out):
        mus = mdp_out[:,:self.num_gaussians*self.nz]

        sigmas= mdp_out[:,self.num_gaussians*self.nz:2*self.num_gaussians*self.nz]
        
        pis = mdp_out[:,-self.num_gaussians:]
        
        
        mus = mus.resize(mus.size(0),self.num_gaussians,self.nz)
        
        sigmas = torch.exp(sigmas)
        sigmas = sigmas.resize(sigmas.size(0),self.num_gaussians,self.nz)
        
        pis = F.softmax(pis,dim=1)
        return mus, sigmas, pis
                        
    
    def forward(self, az):
        

        lstm_out, (self.h_prev,self.c_prev) = self.rnn(az[None,:],(self.h_prev,self.c_prev))
        
        raw_mdp_out = self.mdn_fc(lstm_out[0])
        
        mus, sigmas, pis = self.postproc_mdp_out(raw_mdp_out)
        return mus, sigmas, pis


In [217]:
batch_size = 128

m = M().cuda()

seq_len = 20

In [218]:
z = Variable(torch.Tensor(seq_len,batch_size,m.nz).normal_()).cuda()

a = Variable(torch.Tensor(seq_len,batch_size,3).normal_()).cuda()

az = torch.cat((z,a),dim=2)

In [222]:
outs = []
for azi in az:
    msp = m(azi)
    outs.append(msp)


mus,sigmas,pi = msp

def calc_normal_pdf(z,mus,sigmas):
    num_mixtures = mus.size(1)
    pdfs = []
    for i in range(num_mixtures):
        zmmu = z[0] - mus[:,0]

        zmmu = zmmu[:,:,None]


        sigma = sigmas[:,0]


        exp_term = -0.5 * zmmu.transpose(2,1) @ (zmmu / sigma[:,:,None])

        torch.exp(exp_term[:,0,0])

        torch.prod(sigma,dim=1).size()

        coeff = (1 / (2*np.pi)) * torch.prod(sigma,dim=1)

        pdf = coeff * torch.exp(exp_term[:,0,0])
        pdfs.append(pdf[:,None])
    return torch.cat(pdfs,dim=1)

normals = calc_normal_pdf(z,mus,sigmas)

pdf = (pis * normals).sum(dim=1)

nll = -torch.log(pdf)

nll